**Processing the output of the LODO experiment**

In [ ]:
# Setting the Working Directory to the Notebook's Path

import os
from pathlib import Path

# Get the directory of the current notebook
notebook_path = Path().resolve()
os.chdir(notebook_path)

# Confirm the current working directory
print("Current working directory:", os.getcwd())

In [ ]:
# import libraries

import pandas as pd
import vtk

Prepare the csv files 

*The output of the PhysGNN is shuffled, therefore we need to correct the shufflign and the reorder the csv files*

In [ ]:
# The unshuffled ground truth csv file as reference

input_file_path = 'LODO/final_step/a/output_displacement.csv'  
output_file_path = 'process-compressed/reference.csv'  

# Step 1: Read the CSV file without headers
df = pd.read_csv(input_file_path, header=None)

# Step 2: Rename the columns to 'x', 'y', 'z'
df.columns = ['x', 'y', 'z']

# Step 3: Add an 'id' column starting from 1
df.insert(0, 'id', range(1, len(df) + 1))

# Step 4: Save the updated DataFrame back to a CSV file
df.to_csv(output_file_path, index=False)

print(f"Updated CSV file saved as '{output_file_path}'")

In [ ]:
# The shuffled ground truth csv file as shuffled

input_file_path = 'GNNs-BreastCompression/LODO/Results_LODO/csv/actual_config3.csv'  
output_file_path = 'process-compressed/shuffled.csv'  

# Step 1: Read the CSV file (with current header)
df = pd.read_csv(input_file_path)

# Step 2: Rename the existing columns to 'id', 'x', 'y', 'z'
df.columns = ['id', 'x', 'y', 'z']

# Step 3: Update the 'id' column to be a sequential range starting from 1
df['id'] = range(1, len(df) + 1)

# Step 4: Save the updated DataFrame to a new CSV file
df.to_csv(output_file_path, index=False)

print(f"Updated CSV file saved as '{output_file_path}'")

In [ ]:
# The predicted displacement csv file as prediction

input_file_path = 'GNNs-BreastCompression/LODO/Results_LODO/csv/prediction_config3.csv' 
output_file_path = 'process-compressed/prediction.csv'  

# Step 1: Read the CSV file (with current header)
df = pd.read_csv(input_file_path)

# Step 2: Rename the existing columns to 'id', 'x', 'y', 'z'
df.columns = ['id', 'x', 'y', 'z']

# Step 3: Update the 'id' column to be a sequential range starting from 1
df['id'] = range(1, len(df) + 1)

# Step 4: Save the updated DataFrame to a new CSV file
df.to_csv(output_file_path, index=False)

print(f"Updated CSV file saved as '{output_file_path}'")

Correct shuffling

In [ ]:
# Load the newly uploaded files
new_reference_path = 'process-compressed/reference.csv'
new_shuffled_path = 'process-compressed/shuffled.csv'

new_reference_df = pd.read_csv(new_reference_path)
new_shuffled_df = pd.read_csv(new_shuffled_path)

# Ensure IDs are 1-based in the shuffled DataFrame
new_shuffled_df['id'] = new_shuffled_df.index + 1

# Add a unique identifier to each row to maintain original indices
new_reference_df['ref_unique_id'] = new_reference_df.index
new_shuffled_df['shuffled_unique_id'] = new_shuffled_df.index

# Create a dictionary from the shuffled DataFrame for quick lookups
shuffled_dict = {}
for _, row in new_shuffled_df.iterrows():
    key = (row['x'], row['y'], row['z'])
    if key not in shuffled_dict:
        shuffled_dict[key] = []
    shuffled_dict[key].append((row['id'], row['shuffled_unique_id']))

# Iterate through the reference DataFrame and match rows using the dictionary
matched_rows = []
used_indices = set()

for _, ref_row in new_reference_df.iterrows():
    key = (ref_row['x'], ref_row['y'], ref_row['z'])
    if key in shuffled_dict:
        for shuf_row in shuffled_dict[key]:
            if shuf_row[1] not in used_indices:
                matched_rows.append({
                    'id': shuf_row[0],
                    'x': ref_row['x'],
                    'y': ref_row['y'],
                    'z': ref_row['z'],
                    'original_index_shuffled': shuf_row[1]
                })
                used_indices.add(shuf_row[1])
                break
        else:
            matched_rows.append({
                'id': None,
                'x': ref_row['x'],
                'y': ref_row['y'],
                'z': ref_row['z'],
                'original_index_shuffled': None
            })
    else:
        matched_rows.append({
            'id': None,
            'x': ref_row['x'],
            'y': ref_row['y'],
            'z': ref_row['z'],
            'original_index_shuffled': None
        })

# Create the result DataFrame
result_df = pd.DataFrame(matched_rows)

# Preserve original IDs without changing them to 1-based again
result_df['id'] = result_df['id'].fillna(method='ffill')

# Check if the number of rows in the corrected file matches the reference file
rows_match_final_correct = len(result_df) == len(new_reference_df)

output_path_final_correct = 'process-compressed/corrected_reference.csv'
result_df.to_csv(output_path_final_correct, index=False)

rows_match_final_correct, output_path_final_correct

*Save the ground truth displacements x,y, and z in separate file, and the the Node Indices*

*Based on the Node indices, the order of the predicted displacements will be corrected*

In [ ]:
final_correct_df = pd.read_csv(output_path_final_correct)

# Extract the x, y, z columns
xyz_df = final_correct_df[['x', 'y', 'z']]

indice_df = final_correct_df[['id']]

# Save to a new CSV file without header and index
output_path_xyz = 'process-compressed/xyz_gt.csv'
xyz_df.to_csv(output_path_xyz, index=False, header=False)

indice_path = 'process-compressed/indice.csv'
indice_df.to_csv(indice_path, index=False, header=True)


output_path_xyz
'''

'''# Load the indice and prediction files
prediction_path = 'process-compressed/prediction.csv'

indice_df = pd.read_csv(indice_path)
prediction_df = pd.read_csv(prediction_path)

# Merge the prediction DataFrame with the indice DataFrame on the id column
merged_df = prediction_df.merge(indice_df[['id']], on='id')

# Sort the merged DataFrame based on the order of the id column in the indice DataFrame
sorted_df = merged_df.set_index('id').loc[indice_df['id']].reset_index()

# Extract the x, y, z columns
xyz_df = sorted_df[['x', 'y', 'z']]

# Save the reordered result to a new CSV file without header and index
output_path_xyz_reordered = 'process-compressed/xyz_pred.csv'
xyz_df.to_csv(output_path_xyz_reordered, index=False, header=False)

output_path_xyz_reordered

*Add the displacements to the uncomressed mesh*

In [ ]:
# Read the reordered predicted displacements
diff_csv = 'process-compressed/xyz_pred.csv'

df_diff = pd.read_csv(diff_csv, header=None)

# Load the existing unstructured grid
vtk_filename = 'GNNs-BreastCompression/uncompressed_nrrd/volmesh3.vtk'
reader = vtk.vtkUnstructuredGridReader()
reader.SetFileName(vtk_filename)
reader.Update()

unstructured_grid = reader.GetOutput()

# Step 3: Validate the displacement data and grid size
num_points = unstructured_grid.GetNumberOfPoints()

# Step 4: Create the displacement field data array
displacement_array = vtk.vtkFloatArray()
displacement_array.SetName("displacements")
displacement_array.SetNumberOfComponents(3)  # x, y, z components
displacement_array.SetNumberOfTuples(num_points)

# Step 5: Populate the vtkFloatArray with data from displacement_df
for i in range(num_points):
    displacement = df_diff.iloc[i].values
    displacement_array.SetTuple(i, displacement)

# Step 6: Add the displacement data to the unstructured grid's point data
unstructured_grid.GetPointData().AddArray(displacement_array)
unstructured_grid.GetPointData().SetActiveVectors("displacements")

# Save the updated unstructured grid to a new VTK file
output_vtk = 'process-compressed/volmesh3_physgnn.vtk'
writer = vtk.vtkUnstructuredGridWriter()
writer.SetFileName(output_vtk)
writer.SetInputData(unstructured_grid)
writer.Write()

print(f"Updated VTK file saved to '{output_vtk}'.")

*Get the compressed phantom from the output of Niftysim Docker container*

In [ ]:
case_number = 3

In [ ]:
!python FEA-simualtions/extract_compressed_mesh.py process-compressed/volmesh{case_number}_physgnn.vtk process-compressed/volmesh{case_number}_physgnn_compressed.vtk

*Reconstruct the NRRD image from the compressed phantom mesh of PhysGNN*

In [ ]:
'''Update the -v path !!'''

# Run Docker container with GPU support, mounting a volume, and using an interactive terminal
!docker run --name reconstruct_container --gpus all -v /home/hadeel/GNNs-BreastCompression:/home/data -d reconstruct-image:latest tail -f /dev/null

!docker ps

!docker exec reconstruct_container /bin/bash -c "cd /home/ReconstructImage/release && ./ReconstructImage /home/data/uncompressed_nrrd/volmesh{case_number}.vtk /home/data/process-compressed/volmesh{case_number}_physgnn_compressed.vtk /home/data/uncompressed_nrrd/UncompressedBreast{case_number}_resampled.nrrd /home/data/process-compressed/compressedPhantom{case_number}_physgnn.nrrd"

!docker stop reconstruct_container
!docker rm reconstruct_container